In [ ]:
from config import MINIO_BUCKET_RAW, MINIO_BUCKET_PROCESSED
from etl.stages import stage1, stage2, stage3
from clients.postgres_client import get_postgres_properties
from clients.spark_client import create_spark_session


In [ ]:
spark = create_spark_session()
postgres_props = get_postgres_properties()

In [63]:
raw_paths = stage1(spark, MINIO_BUCKET_RAW, MINIO_BUCKET_PROCESSED)

25/05/04 19:05:26 WARN DAGScheduler: Broadcasting large task binary with size 1004.5 KiB


In [64]:
processed_dfs = stage2(spark, raw_paths, MINIO_BUCKET_PROCESSED)

25/05/04 19:10:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [67]:
stage3(spark, processed_dfs, postgres_props)